In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement

Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# View first row of data for measurement table
first_row_measurement = session.query(Measurement).first()
first_row_measurement.__dict__

In [ ]:
# View first row of data for station table
first_row_station = session.query(Station).first()
first_row_station.__dict__

# Exploratory Precipitation Analysis

HOW DO WE DO THIS? WHAT STATION DO WE USE? IT DOESN'T MAKE SENSE WHEN WE TAKE AN AVERAGE BY DATE?

In [ ]:
session.query(Station.name,Station.station).group_by(Station.name).all()

In [ ]:
# Find the most recent date in the data set.
first_date = session.query(Measurement.date).\
    group_by(Measurement.date).\
    order_by(Measurement.date.desc()).first()
first_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
year_ago = dt.date(2017,8,23) - dt.timedelta(days=365)
print(year_ago)

# Perform a query to retrieve the data and precipitation scores
precipitation = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= year_ago).all()
precipitation

# Save the query results as a Pandas DataFrame and set the index to the date column
precipitation_df = pd.DataFrame(precipitation,columns=['date','precipitation'])
precipitation_df.set_index(['date'],inplace=True)

# Sort the dataframe by date
precipitation_df.sort_index(inplace=True)

# Use Pandas Plotting with Matplotlib to plot the data
precipitation_df.plot(kind='line',rot=90,figsize=(9,6));
plt.ylabel('Precipitation (inches)');
plt.xlabel('Date');
plt.legend(loc='upper right');
plt.xlim(0,len(precipitation_df))

x_axis = np.arange(0,len(precipitation_df),100)
x_axis


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precipitation_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
session.query(Measurement).group_by(Measurement.station).count()

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
session.query(Measurement.station,func.count(Measurement.prcp)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.prcp).desc()).all()


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
min_temp = session.query(func.min(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519281').first()

max_temp = session.query(func.max(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519281').first()

mean_temp = session.query(func.avg(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519281').first()

print(f'Min temp: {min_temp}, Max temp: {max_temp}, Mean temp: {mean_temp}')


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
last_date_9281 = session.query(Measurement.date).\
    filter(Measurement.station == 'USC00519281').\
    order_by(Measurement.date.desc()).first()

year_ago_9281 = dt.date(2017,8,18) - dt.timedelta(days=365)

temps_9281 = session.query(Measurement.tobs).\
    filter(Measurement.date >= '2016-08-18').\
    filter(Measurement.station == 'USC00519281').all()

temps_9281 = pd.Series(temps_9281)
temps_9281 = temps_9281.astype(int)


# Close session

In [ ]:
# Close Session
session.close()